# **Hugging Face Whisper Small Model Train and Fine-Tuning**

In [ ]:
'''
prevent the Colab runtime from disconnecting & automatically click the connect button in Google Colab at regular intervals (every minute)

copy below javascript --> in colab, right click and select Inspect --> paste in console


function ConnectButton(){
     console.log("Connect pushed");
     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
'''

## **Hugging Face Login**

In [ ]:
# access and work with models that are stored on the Hugging Face Hub.
# generate access token with 'Write' access from your Huggingface account & use it to login in colab.

from huggingface_hub import notebook_login

notebook_login()

## **Install Required Libraries**

In [ ]:
!pip install --upgrade pip
!pip install --upgrade datasets transformers accelerate soundfile librosa evaluate jiwer tensorboard

'''
datasets - specifically designed for loading datasets from the Hugging Face Datasets Hub,
           which is a centralized repository of various datasets for natural language processing tasks.

Transformers  : Access to large collection of pre-trained models for various NLP tasks

accelerate - it is a Python library developed by Hugging Face, purpose is to speed up the training and inference process of deep learning models.

soundfile - helps in reading and writing sound files

librosa - It can be helpful for extracting features from audio

evaluate - evaluating and comparing models and reporting their performance easier and more standardized

jiwer - calculating the Word Error Rate (WER)

tensorboard - It is a visualization tool included with TensorFlow. It helps in monitoring and debugging your machine learning models.
'''

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## **Data Collection**

In [ ]:
# DatasetDict - It is designed to organize and manage multiple datasets effectively - manage train and test dataset are clear

from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()    # create empty DatasetDict

common_voice

DatasetDict({
    
})

In [ ]:
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="train+validation", use_auth_token=True)
common_voice["train"]

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 3927
})

In [ ]:
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="test", use_auth_token=True)
common_voice["test"]

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 1816
})

In [ ]:
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 3927
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1816
    })
})


In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3927
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1816
    })
})

## **Data preprocessing**

### **Feature Extractor**

In [ ]:
# WhisperFeatureExtractor - It takes raw audio input and transforms it into a format that the model can understand the patterns

from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
feature_extractor

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

### **Tokenizer - Text Encoding and Decoding**

In [ ]:
# WhisperTokenizer -  The process of splitting long text into small chunks, where each chunk is called a token
#                     and is assigned a numerical representation, is essential for the computer to understand and process the information.

from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Marathi", task="transcribe")

In [ ]:
# input sentence in Marathi taken from the common voice dataset
text = common_voice["train"][0]["sentence"]

# encode (machine readable format) --> text encoded to numerical numbers
encoded_text = tokenizer(text).input_ids

# decode (human readable format) --> token ID convert to original text with special token
decoded_with_special = tokenizer.decode(token_ids = encoded_text,
                                        skip_special_tokens = False)

# decode the token ID convert to original text w/o special token
decoded_text = tokenizer.decode(token_ids = encoded_text,
                                skip_special_tokens = True)

print(f'Text          : {text}')
print(f'Encode Text   : {encoded_text}')
print(f'Decode s.token: {decoded_with_special}')
print(f'Decoded Text  : {decoded_text}')

Text          : आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.
Encode Text   : [50258, 50320, 50359, 50363, 3941, 228, 3941, 230, 3941, 248, 21981, 8485, 228, 3941, 250, 17937, 25411, 3941, 103, 3941, 96, 8485, 113, 17937, 3941, 95, 36158, 8485, 248, 17937, 46758, 46758, 21981, 11, 8485, 97, 45938, 36158, 3941, 114, 31881, 48449, 3941, 98, 31881, 44500, 31881, 8485, 101, 31881, 3941, 253, 8485, 244, 17937, 36158, 3941, 103, 31881, 36158, 35082, 17937, 3941, 114, 31881, 8485, 251, 17937, 46758, 31881, 13, 50257]
Decode s.token: <|startoftranscript|><|mr|><|transcribe|><|notimestamps|>आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.<|endoftext|>
Decoded Text  : आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.


### **Processor**

In [ ]:
# WhisperProcessor - prepares your Marathi audio data so that it can be understood and transcribed by the Whisper model.
#                    It's like a translator between your raw audio and the model.

from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Marathi", task="transcribe")

### **Sampling Rate Conversion**

In [ ]:
common_voice["train"][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f7e1ef6a2d14f20194999aad5040c5d4bb3ead1377de3e1bbc6e9dba34d18a8a/mr_train_0/common_voice_mr_30585613.mp3',
  'array': array([ 1.13686838e-13, -1.42108547e-13, -1.98951966e-13, ...,
          4.83472422e-06,  3.54798703e-06,  1.63231743e-06]),
  'sampling_rate': 48000},
 'sentence': 'आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.'}

In [ ]:
'''
convert the audio data sampling rate from 48000 HZ to 16000 HZ, ensuring compatibility with downstream tasks.
This standardization is essential for consistent processing and analysis in speech to text.
'''

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3927
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1816
    })
})

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f7e1ef6a2d14f20194999aad5040c5d4bb3ead1377de3e1bbc6e9dba34d18a8a/mr_train_0/common_voice_mr_30585613.mp3', 'array': array([-3.06954462e-12, -3.63797881e-12, -4.54747351e-12, ...,
       -7.74800901e-06, -1.74738125e-06,  4.36312439e-06]), 'sampling_rate': 16000}, 'sentence': 'आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.'}


In [ ]:
print(common_voice["train"][0]['audio'])

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f7e1ef6a2d14f20194999aad5040c5d4bb3ead1377de3e1bbc6e9dba34d18a8a/mr_train_0/common_voice_mr_30585613.mp3', 'array': array([-3.06954462e-12, -3.63797881e-12, -4.54747351e-12, ...,
       -7.74800901e-06, -1.74738125e-06,  4.36312439e-06]), 'sampling_rate': 16000}


In [ ]:
print(f"train_path          = {common_voice['train'][0]['audio']['path']}")
print(f"train_array         = {common_voice['train'][0]['audio']['array']}")
print(f"train_sampling_rate = {common_voice['train'][0]['audio']['sampling_rate']}")
print(f"train_sentence      = {common_voice['train'][0]['sentence']}")

train_path          = /root/.cache/huggingface/datasets/downloads/extracted/f7e1ef6a2d14f20194999aad5040c5d4bb3ead1377de3e1bbc6e9dba34d18a8a/mr_train_0/common_voice_mr_30585613.mp3
train_array         = [-3.06954462e-12 -3.63797881e-12 -4.54747351e-12 ... -7.74800901e-06
 -1.74738125e-06  4.36312439e-06]
train_sampling_rate = 16000
train_sentence      = आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.


### **Dataset Preparation**

In [ ]:
'''
Log-Mel features (audio):

Mel refers Mel-frequency, a perceptual scale that mimics how humans hear different pitches.

Log-Mel features involve measuring the audio signal's pitches using Mel scale and taking 'logarithm' of those values.

widely used in speech and audio processing, particularly in speech recognition.
'''

In [ ]:
common_voice

In [ ]:
# Audio and Text Dataset Preprocessing for Machine Learning

def dataset_preparation(batch):

    # raw audio data transform into log-Mel features
    batch["input_features"] = feature_extractor(batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"]).input_features[0]

    # sentence encode to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    return batch

In [ ]:
common_voice = common_voice.map(dataset_preparation, remove_columns=['audio', 'sentence'], num_proc=1)  # number of parallel processes = 1
common_voice

Map:   0%|          | 0/3927 [00:00<?, ? examples/s]

Map:   0%|          | 0/1816 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3927
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1816
    })
})

In [ ]:
"""
step 1 [batch - raw audio input]:

{'audio': {'path': '/root/.cache/......./mr_train_0/common_voice_mr_30585613.mp3',
           'array': array([-3.06954462e-12, -3.63797881e-12, ... 4.36312439e-06]),
           'sampling_rate': 16000},

'sentence': 'आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.'}

===========================================================================================================================

step 2 (batch['audio']):

{'path': '/root/.cache/......./mr_train_0/common_voice_mr_30585613.mp3',
 'array': array([-3.06954462e-12, -3.63797881e-12, ... 4.36312439e-06]),
 'sampling_rate': 16000}

===========================================================================================================================

step 3 (feature extract from raw audio):

{'input_features': [array([[-0.57933486, -0.57933486, ..., -0.57933486, -0.57933486],
                           ...,
                           [-0.57933486, -0.57933486, ..., -0.57933486, -0.57933486]],
                   dtype=float32)]}

===========================================================================================================================

step 3.1 (batch['input_features']):

{'audio': {'path': '/root/.cache/......./mr_train_0/common_voice_mr_30585613.mp3',
           'array': array([-3.06954462e-12, -3.63797881e-12, ... 4.36312439e-06]),
           'sampling_rate': 16000},

'sentence': 'आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.',

'input_features': array([[-0.57933486, -0.57933486, ..., -0.57933486, -0.57933486],
                           ...,
                           [-0.57933486, -0.57933486, ..., -0.57933486, -0.57933486]],
                   dtype=float32)}

===========================================================================================================================

step 4 (sentence encode to token ids):

{'input_ids'     : [50258, 50320, 50359, 50363, 3941, 228, ... 31881, 8485, 251, 17937, 46758, 31881, 13, 50257],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

===========================================================================================================================

step 5 (batch['labels']):

{'audio': {'path': '/root/.cache/......./mr_train_0/common_voice_mr_30585613.mp3',
           'array': array([-3.06954462e-12, -3.63797881e-12, ... 4.36312439e-06]),
           'sampling_rate': 16000},

'sentence': 'आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.',

'input_features': array([[-0.57933486, -0.57933486, ..., -0.57933486, -0.57933486],
                           ...,
                         [-0.57933486, -0.57933486, ..., -0.57933486, -0.57933486]],
                   dtype=float32),

{'labels': [50258, 50320, 50359, 50363, 3941, 228, ... 31881, 8485, 251, 17937, 46758, 31881, 13, 50257]}

===========================================================================================================================

Finally columns ['audio', 'sentence'] are removed and only ['input_features', 'labels'] columns are in the custom_voice dataset

"""


### **Data Collector**

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": i["input_features"]} for i in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": i["labels"]} for i in features]

        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
data_collator

In [ ]:
import evaluate

metric = evaluate.load("wer")
metric

EvaluationModule(name: "wer", module_type: "metric", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Compute WER score of transcribed segments against references.

Args:
    references: List of references for each speech input.
    predictions: List of transcriptions to score.
    concatenate_texts (bool, default=False): Whether to concatenate all input texts or compute WER iteratively.

Returns:
    (float): the word error rate

Examples:

    >>> predictions = ["this is the prediction", "there is an other sample"]
    >>> references = ["this is the reference", "there is another one"]
    >>> wer = evaluate.load("wer")
    >>> wer_score = wer.compute(predictions=predictions, references=references)
    >>> print(wer_score)
    0.5
""", stored examples: 0)

In [ ]:
def compute_metrics(pred):

    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
"""
PEFT - Parameter Efficient Fine-Tuning

In this method certain layers of the model are freezed during fine-tuning in LLM.

This approach updates only a small subset of the models parameters. Since other parameters are freezed,
it requires less memory, resulting in faster training and making it more cost-effective compared to other methods.


LoRA - Low-Rank Adapters

This method calculates and print the
  - total number of trainable parameters
  - total number of all parameters
  - percentage of trainable parameters

This information provides an overview of the models complexity and resource requirements for training.

"""

In [ ]:
"""
!pip install peft

from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model

model = prepare_model_for_int8_training(model)

def make_inputs_require_grad(module, input, output):
      output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

"""

In [ ]:
# lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

# # purpose is to modify the original model based on the specified LORA configuration.
# model = get_peft_model(model, lora_config)

# model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 243,504,384 || trainable%: 0.7266694631666262


**Note**:

*IndexError: Invalid key: 3403 is out of bounds for size 0 -----> error faced during the model training.*

*Thats why PEFT & LORA methods has beed skipped in the fine-tuning process.*

## **Model Training**

In [ ]:
"""
Note: Model Fine-tuning and training has not worked properly. Because during the training process, faced the issue of

    - CUDA Setup failed despite GPU being available
    - NotImplementedError: Cannot copy out of meta tensor; no data!

So change the training args parameters and calculate the loss and error at different stpes [25, 50, 100].

"""

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
                                          output_dir="./whisper-small-mr",  # change to a repo name of your choice
                                          per_device_train_batch_size=8,    # 8 samples pocessed in each training batch
                                          gradient_accumulation_steps=2,    # increase by 2x for every 2x decrease in batch size
                                          learning_rate=2e-2,               # learning_rate = 0.02
                                          warmup_steps=50,                  # learning rate graduqally incresed from 0 to 50 and after 50 it should same learning rate 50-100 steps
                                          max_steps=100,                    # Maximum number of training steps.
                                          gradient_checkpointing=True,      # it store only necessary results to save memory
                                          fp16=True,                        # 16-bit floating-point precision for certain operations & speeding up training
                                          evaluation_strategy="steps",
                                          per_device_eval_batch_size=8,     # evaluation batch size
                                          predict_with_generate=True,       # in evaluation will predict outcomes and also generate new sequences of text
                                          generation_max_length=225,        # Maximum length of generated sequences during evaluation.
                                          save_steps=50,                    # saving models in every 50 steps
                                          eval_steps=25,                    # evaluate the model during training in each 25 steps
                                          logging_steps=25,                 # logging information automatically stored by huggingface transformer library
                                          report_to=["tensorboard"],        # training logs should be reporte to TensorBoard
                                          load_best_model_at_end=True,
                                          metric_for_best_model="wer",
                                          greater_is_better=False,          # error metrics like Word Error Rate (WER) or similar, where lower values indicate better performance
                                          push_to_hub=True,
                                          push_to_hub_model_id="whisper-small-mr"
                                      )
training_args

Seq2SeqTrainingArguments(output_dir='./whisper-small-mr', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=2, eval_accumulation_steps=None, eval_delay=0, learning_rate=0.02, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=100, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=50, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='./whisper-small-mr/runs/Dec30_10-49-56_0db79d94c419', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=25, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=50

In [ ]:
# manage the training process of sequence-to-sequence model

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
                          args=training_args,
                          model=model,
                          train_dataset=common_voice["train"],
                          eval_dataset=common_voice["test"],
                          data_collator=data_collator,
                          compute_metrics=compute_metrics,
                          tokenizer=processor.feature_extractor)
trainer

In [ ]:
trainer.train()

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
25,19.471000,35.392952,100.000000


Step,Training Loss,Validation Loss,Wer
25,19.471000,35.392952,100.000000


### **Model Max_steps = 50**

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
                                          output_dir="./whisper-small-mr",
                                          per_device_train_batch_size=8,
                                          gradient_accumulation_steps=2,
                                          learning_rate=2e-2,
                                          warmup_steps=25,
                                          max_steps=50,
                                          gradient_checkpointing=True,
                                          fp16=True,
                                          evaluation_strategy="steps",
                                          per_device_eval_batch_size=8,
                                          predict_with_generate=True,
                                          generation_max_length=225,
                                          save_steps=50,
                                          eval_steps=50,
                                          logging_steps=50,
                                          report_to=["tensorboard"],
                                          load_best_model_at_end=True,
                                          metric_for_best_model="wer",
                                          greater_is_better=False,
                                          push_to_hub=True,
                                          push_to_hub_model_id="whisper-small-mr"
                                      )
training_args

Seq2SeqTrainingArguments(output_dir='./whisper-small-mr', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=2, eval_accumulation_steps=None, eval_delay=0, learning_rate=0.02, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=50, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=25, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='./whisper-small-mr/runs/Dec30_23-11-27_44c3f51c7637', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=50, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=50,

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
                          args=training_args,
                          model=model,
                          train_dataset=common_voice["train"],
                          eval_dataset=common_voice["test"],
                          data_collator=data_collator,
                          compute_metrics=compute_metrics,
                          tokenizer=processor.feature_extractor)
trainer

In [ ]:
trainer.train()

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
50,72.229600,153.046295,100.000000


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=50, training_loss=72.2296435546875, metrics={'train_runtime': 2513.6542, 'train_samples_per_second': 0.318, 'train_steps_per_second': 0.02, 'total_flos': 2.30868320256e+17, 'train_loss': 72.2296435546875, 'epoch': 0.2})

### **Model Max_steps = 100**

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
                                          output_dir="./whisper-small-mr",
                                          per_device_train_batch_size=8,
                                          gradient_accumulation_steps=2,
                                          learning_rate=2e-2,
                                          warmup_steps=50,
                                          max_steps=100,
                                          gradient_checkpointing=True,
                                          fp16=True,
                                          evaluation_strategy="steps",
                                          per_device_eval_batch_size=8,
                                          predict_with_generate=True,
                                          generation_max_length=225,
                                          save_steps=100,
                                          eval_steps=100,
                                          logging_steps=100,
                                          report_to=["tensorboard"],
                                          load_best_model_at_end=True,
                                          metric_for_best_model="wer",
                                          greater_is_better=False,
                                          push_to_hub=True,
                                          push_to_hub_model_id="whisper-small-mr"
                                      )
training_args

Seq2SeqTrainingArguments(output_dir='./whisper-small-mr', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=2, eval_accumulation_steps=None, eval_delay=0, learning_rate=0.02, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=100, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=50, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='./whisper-small-mr/runs/Dec30_13-29-34_b7eabe5028e2', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=100, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=1

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
                          args=training_args,
                          model=model,
                          train_dataset=common_voice["train"],
                          eval_dataset=common_voice["test"],
                          data_collator=data_collator,
                          compute_metrics=compute_metrics,
                          tokenizer=processor.feature_extractor)
trainer

In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
100,84.853600,110.923683,100.000000


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=100, training_loss=84.853642578125, metrics={'train_runtime': 2762.6552, 'train_samples_per_second': 0.579, 'train_steps_per_second': 0.036, 'total_flos': 4.61736640512e+17, 'train_loss': 84.853642578125, 'epoch': 0.41})

## **Trained Model Push to Hugging Face Hub**

In [ ]:
peft_model_id = "gopiashokan/whisper-small-mr"

model.push_to_hub(peft_model_id)

 CommitInfo(commit_url='https://huggingface.co/gopiashokan/whisper-small-mr/commit/6d57dc68223efe0e33923f9753be04039359e302', commit_message='Upload
 WhisperForConditionalGeneration', commit_description='', oid='6d57dc68223efe0e33923f9753be04039359e302', pr_url=None, pr_revision=None, pr_num=None)


## **Conclusion**

In [ ]:
"""
The Whisper-large-v3 model exhibits superior accuracy when compared to other models. However, a significant challenge arises
as training the model with a custom dataset and fine-tuning in Google Colab leads to automatic crashes beyond a certain limit.
To overcome this limitation, we opt for the Whisper-small model for training and fine-tuning with a custom dataset.

"""

**Note**: *Performance of the Whisper-small model (WER = 3.38) is comparatively poorer than the Whisper-large-v3 model (WER = 0.74)*

In [ ]:
"""
We trained the model with a limit of only 100 steps due to the constraints of Google Colab, as it supports training up to this
level before encountering GPU limitations and automatic crashes. To further enhance the model's understanding of patterns and
improve prediction performance, additional training steps are required. However, this extends beyond the capabilities of the
Google Colab, necessitating alternative resources or platforms for extended training.

"""

In [ ]:
"""
In the future, plan to leverage cloud platforms such as AWS or Azure to train the Whisper-large-v3 model with a custom dataset and
conduct fine-tuning. This approach will provide the necessary resources and scalability to train the model extensively, allowing us
to reach maximum efficiency and enhance its predictive capabilities.

"""